# Aditional examples implementing the Ideal Flash module.
---

This notebook will show you how to change some of the parameters on the *example.py* file. First we will solve the same example but making use of **Jupyter Notebooks**. After that, a couple of additional example extract them from the book *Separation Process Engineering* by *Phillip C. Wankat*, Third edition will be solve as well.

In [ ]:
# Importing the modules
from streams.substance import Substance
from streams.stream import Stream
from streams.parameter import Parameter
from opus.flash import Flash

## Example 1
---

A flash chamber operating at 50°C and 200 kPa is separating 1000 kmol/h of afeed that is 30 mol% propane, 10 mol% n-butane, 15 mol% n-pentane and 45 mol%n-hexane. Find the product compositions and flow rates

![Diagram of the problem](https://oscarcontrerasnavas.github.io/assets/img/idealDistillation/processDiagram.jpg)

In [ ]:
# Define substances
propane = Substance('Propane')
butane = Substance('Butane')
pentane = Substance('Pentane')
hexane = Substance('Hexane')
substances = [propane, butane, pentane, hexane]
composition = [0.3, 0.1, 0.15, 0.45]

In [ ]:
# Define Parameters
temperature = Parameter('Temperature', 323, 'K')
pressure = Parameter('Pressure', 200, 'kPa')
flow_rate = Parameter('Mass Flow Rate', 1000,'kmol/h')

In [ ]:
#Define Streams
inlet = Stream('Inlet', substances, flow_rate, composition, pressure, temperature)
vapor = Stream('Vapor', substances)
liquid = Stream('Liquid', substances)

In [ ]:
# Define a Flash
flash = Flash('V-101', inlet, vapor, liquid)
flash.solve()
to_print() 

Our solution is $V = 503.8$ kmol/h and $L = 496.2$ kmol/h and the book solution using the DePriester charts is $V = 510$ kmol/h and $L = 490$ kmol/h. Preaty close for an ideal method.

## Example 2
---
We wish to flash distill a feed that is 10 mol% propane, 30 mol% n-butane, and 60 mol% nhexane. Feed rate is 10 kmol/h, and drum pressure is 200 kPa. We desire a liquid that is 85 mol% n-hexane. Find T and V/F.

In [ ]:
# Define substances
propane = Substance('Propane')
butane = Substance('Butane')
hexane = Substance('Hexane')
substances = [propane, butane, hexane]
composition = [.10, .30, .60]

In [ ]:
# Define Parameters
temperature = Parameter('Temperature', 300, 'K') # Test 1
pressure = Parameter('Pressure', 200, 'kPa')
flow_rate = Parameter('Mass Flow Rate', 10,'kmol/h')

In [ ]:
#Define Streams
inlet = Stream('Inlet', substances, flow_rate, composition, pressure, temperature)
vapor = Stream('Vapor', substances)
liquid = Stream('Liquid', substances)

In [ ]:
# Define a Flash
flash = Flash('V-101', inlet, vapor, liquid)
flash.solve()
to_print()

Now we will change the temperature until we reach a  $x_{hexane} = 0.85$ molar fraction

In [ ]:
error = 1
desired = .85
i=0
while error >= 1e-3:
    i+=1
    t = 300 + 0.1*i
    flash.inlet.temperature.value = t
    flash.solve()
    current = flash.liquid.compositions["hexane"]
    error = abs(current - desired)    
    
# check results    
print("T = {0:.2f} {1}\n".format(flash.inlet.temperature.value, flash.inlet.temperature.units))    
to_print()    

Our solution is $T = 336.10$ K and $V/F = 0.51$ and the book solution using the DePriester charts is $T = 338.6$ K and $V/F = 0.57$. Preaty close for an ideal method.

## Example 3
---
A flash drum operating at 300 kPa is separating a mixture that is fed in as 40 mol% isobutane, 25% n-pentane, and 35% n-hexane. We wish a 90% recovery of n-hexane in the liquid (that is, 90% of the n-hexane in the feed exits in the liquid product). F = 1000 kmol/h. Find T, xj, yj, V/F.

In [ ]:
# Define substances
isobutane = Substance('Isobutane')
pentane = Substance('Pentane')
hexane = Substance('Hexane')
substances = [isobutane, pentane, hexane]
composition = [.40, .25, .35]

In [ ]:
# Define Parameters
temperature = Parameter('Temperature', 330, 'K') # Test 1
pressure = Parameter('Pressure', 300, 'kPa')
flow_rate = Parameter('Mass Flow Rate', 1000,'kmol/h')

In [ ]:
#Define Streams
inlet = Stream('Inlet', substances, flow_rate, composition, pressure, temperature)
vapor = Stream('Vapor', substances)
liquid = Stream('Liquid', substances)

In [ ]:
# Define a Flash
flash = Flash('V-101', inlet, vapor, liquid)
flash.solve()
to_print()
print("\nLx_hex = {0:.2f} kmolh".format(flash.liquid.flow_rate.value * flash.liquid.compositions["hexane"]))

Now we need to loop over the `Temperature` until we reach 

$$Lx_{\text{hexane}} = 0.9Fz_{\text{hexane}}$$

or 

$$Lx_{\text{hexane}} = 315 \ \text{kmol/h}$$

In [ ]:
error = 100
desired = 315
i=0
while error >= 1:
    i+=1
    t = 330 + 0.1*i
    flash.inlet.temperature.value = t
    flash.solve()
    current = flash.liquid.compositions["hexane"] * flash.liquid.flow_rate.value
    error = abs(current - desired)    
    
# check results    
print("T = {0:.2f} {1}\n".format(flash.inlet.temperature.value, flash.inlet.temperature.units))    
to_print()  
print("\nLx_hex = {0:.2f} kmolh".format(flash.liquid.flow_rate.value * flash.liquid.compositions["hexane"]))

Our solution is $T = 330.70$ K and $V/F = 0.317$ and the book solution using the DePriester charts is $T = 330.15$ K and $V/F = 0.293$. Preaty close for an ideal method.

In [ ]:
def to_print():
    
    # For inlet
    print(
        "F = {0} {1}".format(flash.inlet.flow_rate.value, flash.inlet.flow_rate.units) 
    )
    for tag, substance in flash.inlet.substances.items():
        print(
            "z_{0} = {1}".format(substance.tag, flash.inlet.compositions[tag])
        )
        
    # For vapor
    print("")
    print(
        "V = {0:.1f} {1}".format(flash.vapor.flow_rate.value, flash.vapor.flow_rate.units) 
    )
    for tag, substance in flash.vapor.substances.items():
        print(
            "y_{0} = {1:.3f}".format(substance.tag, flash.vapor.compositions[tag])
        )
        
    # For liquid
    print("")
    print(
        "L = {0:.1f} {1}".format(flash.liquid.flow_rate.value, flash.liquid.flow_rate.units) 
    )
    for tag, substance in flash.liquid.substances.items():
        print(
            "x_{0} = {1:.3f}".format(substance.tag, flash.liquid.compositions[tag])
        )